In [7]:
import tweepy
import datetime
import time
import json
import sys
from tweepy import OAuthHandler, API

class TweetWorker:
    # OAuth
    __consumer_key = "dL2ZAItvpFLiCULxa8Y7E5Utg"
    __consumer_secret = "hnLfXSmCTNSAJ2rsU9QOAvUFG159wysgGwy38sQyt0XKDte70M"
    __access_token = "709584127-zufPerRgSs4TDMscyfon47CIsN2AuXapBU8XEhty"
    __access_token_secret = "wzpAq42SKJP4XpqXqWAfiNsAJmXKjN2AQSeP51BvIETE2"    
    
    
    ####
    def __init__(self, queue):
        self.log('\n'*2 + "="*110)
        self.log("\n__Initialized__ @" + time.asctime(time.gmtime()) + "\n")
        self.log("="*110 + '\n'*2)
        auth = OAuthHandler(self.__consumer_key, self.__consumer_secret)
        auth.set_access_token(self.__access_token, self.__access_token_secret)
        self.api = API(auth)#, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) 
            # TODO: test if auto waits and resumes
        
        self.last_min_id = None
        self.last_max_id = None
        self.request_count = 0  # For fun, keep track of number of requests.
        self.queue = queue
        
    
    def get_until_date(self, q, shift = 8):
        ''' Get the oldest possible tweet date from now.
                (Twitter standard -> 7-day-history -> can get tweets until 6 days ago)
                (In fact, can actually searched 8 days ago)
            Input: q = <'query_string'>
                   shift = <num of days before today>
            Return: The earliest date that API.search returns a result in "YYYY-MM-DD"
        '''
        today = datetime.date.today()
        
        while True:      
            shift_date = timedelta(days = shift) 
            until = (today - shift_date).strftime("%Y-%m-%d")
            
            self.request_count += 1
            self.log("[get_until_date] Request Count: "+str(self.request_count))
            
            if api.search(q = q, until = until, count = 1):
                print(until)
                return until
            shift -= 1
            
    
    def get_oldest_tweet_id(self, q):
        ''' Get the oldest possible tweet id for query q.
            (Twitter standard -> 7-day-history)
        '''        
        self.request_count += 1
        self.log("[get_oldest_tweet_id] Request Count: "+str(self.request_count))
        
        until = self.get_until_date(q)
        res = self.api.search(q = q, until = until)
        
        id = res[-1]._json['id']
        print("Oldest ID: ", id)
        return id
    
    def get_latest_tweet_id(self, q):
        ''' Get the latest possible tweet id for query q
        '''
        self.request_count += 1
        self.log("[get_latest_tweet_id] Request Count: "+str(self.request_count))
        
        # Will Retry, until get a tweet, or trial upto 10 times
        count = 0
        while count <= 10:
            count += 1
            self.log("[get_latest_tweet_id] Getting the latest ")
            try: 
                res = self.api.search(q = q)
                id = res[0]._json['id']
                return id
            except:
                print("Cannot find tweets... Retry in 10 sec")
                time.sleep(10)
                
        self.log("[get_latest_tweet_id] Cannot find tweets after 10 trials...")
        self.log("[get_latest_tweet_id] Exit system.")
        sys.exit(0)
        return None


    def search_tweets(self, q, lang="en", count=100):
        ''' 
        '''
        self.request_count += 1
        self.log("[search_tweets] Request Count: "+str(self.request_count))
        
        # API.search(q[, geocode][, lang][, locale][, result_type]
          # [count (max=100)][, until][, since_id][, max_id])
        print("[search_tweets] Searching latest id: ", self.last_min_id)
        result_status = self.api.search(
                                       q = q, 
                                       count=100,
                                       max_id = self.last_min_id,
                                       result_type = 'recent' )
        
        # Update last_min_id for next round of search as 'max_id'.
        self.last_min_id = result_status[-1]._json['id']
        
        
        return result_status  # can be empty, []
        
    def stream_tweets():
        return
    
    def rate_limit_handle(self, func, q):
        ''' Try to use API. Wait if rate limit reached. 
        '''
        try:
            ret = func(q=q)
            return ret

        # Catch Twitter API rate limit exception and wait for 15 minutes
        except tweepy.RateLimitError:
            print('Hit Twitter API rate limit.')
            for i in range(3, 0, -1):
                print("Wait for {} mins.".format(i * 5))
                time.sleep(5 * 60)
            print("Finished Waiting.")

        # Catch any other Twitter API exceptions
        except tweepy.error.TweepError:
            print('\nCaught TweepError exception')
            
    
    def log(self, message):
        """ Logs all operating message in to txt file.
            Only for Debugging use.
        """
        print(message)
        with open("log_tweetWorker.txt", "a") as f:
            f.write(message+'\n')

    
    def run_search(self, q):
        ''' This is the main run. 
            Continously search tweets in chronical order, and save every tweet into a json
              file with tweet id as name.
        '''
        # Step 1: Get the min_id for 'since_id' in api.search()

        self.last_max_id = self.get_latest_tweet_id(q=q)
        self.last_min_id = self.last_max_id
        self.log("[MAIN RUN] last_max_id: "+ str(self.last_max_id))
        
        # Step 2: 
        round_count = 0
    
        for i in range(50):
            self.log("[MAIN RUN] Current round: "+str(round_count))
            round_count += 1
            
            func = self.search_tweets
            result_status = self.rate_limit_handle(func, q)

            # Save the status:
            result_count = 0
            if result_status:
                
                for status in result_status:
                    self.log("On status #: "+str(result_count))
                    
                    result_count += 1
                    
                    status_json = status._json
                    status_id = status_json['id']
                    
                    self.log("  -- id: "+str(status_id))
                    
                    # option1: save to json
#                     with open('json_cache/' + str(status_id) + '.json', 'w') as f:
#                         json.dump(status_json, f)
                        
                    # option2: save to queue
                    self.queue.put(status_json)
                    
                        

In [8]:
import couchdb
import os

class CouchdbWorker:
    """ save json documents to db "tweet_json"
    """
    __user = "admin"
    __secret = "admin"
    db_name = "tweet_json"
    
    
    
    def __init__(self, queue):
        """ queue: TweetWorker PUT status to queue, CouchdbWorker GET and save to db
        """
        login = self.__user + ':' + self.__secret
        url = "http://" + login + "@localhost:5984/"
        
        self.couch = couchdb.Server(url)
        self.db = self.setup_db(CouchdbWorker.db_name)
        self.queue = queue
        self.save_count = 0  # count the number of success SAVE to db
        
    def log(self, message):
        """ Logs all operating message in to txt file.
            Only for Debugging use.
        """
        print(message)
        with open("log_couchWorker.txt", "a") as f:
            f.write(message+'\n')
        
    def setup_db(self, name):
        ''' If db with <name> not exist, create one.
        '''
        try:
            db = self.couch[name]
        except couchdb.ResourceNotFound as e:
            print("Database", name, "is not found. Creating one instead.")
            db = self.couch.create(name)
        return db
    
    def save_doc(self, db, status_dict):
        ''' Save a dictionary object into database.
            Return <document id>, <document rev>
        '''
        try:
            status_id = status_dict['id_str']
            status_dict['_id'] = status_id
            tmp_id, tmp_rev = db.save(status_dict)
            self.save_count += 1
            print("Successful save: ", self.save_count)
            return tmp_id, tmp_rev
        
        except couchdb.ResourceConflict as e:
            print("Database already have document:", status_id)
        except :
            print("Unknown Error.")
    
        return None, None
    
    def run_save(self):
        """ Continuously running to save doc to db
        """
        
        # IF queue is still empty after waiting for 5*15 mins, exit
        count = 0
        while count < 5:
            count += 1
            if not self.queue.empty():
                count = 0
                print("Not empty,", end=' ')
                doc = self.queue.get()
                self.save_doc(self.db, doc)
                continue
                
            # IF queue is empty, TwitterAPI may have reached rate limit, pause 15min
            self.log("."*40 + "waiting for TweetWorker..")
            time.sleep(15 * 60)
            


In [9]:
from queue import Queue 
queue = Queue()
tw = TweetWorker(queue)
cw = CouchdbWorker(queue)




__Initialized__ @Sun May  3 13:05:19 2020





In [10]:
tw.run_search("stayhome")

[get_latest_tweet_id] Request Count: 1
[get_latest_tweet_id] Getting the latest 
[MAIN RUN] last_max_id: 1256932874848055296
[MAIN RUN] Current round: 0
[search_tweets] Request Count: 2
[search_tweets] Searching latest id:  1256932874848055296
On status #: 0
  -- id: 1256932874848055296
On status #: 1
  -- id: 1256932874512556035
On status #: 2
  -- id: 1256932874290413570
On status #: 3
  -- id: 1256932873774325761
On status #: 4
  -- id: 1256932872780492802
On status #: 5
  -- id: 1256932871710797824
On status #: 6
  -- id: 1256932870121123841
On status #: 7
  -- id: 1256932869449977856
On status #: 8
  -- id: 1256932868070072320
On status #: 9
  -- id: 1256932868007182339
On status #: 10
  -- id: 1256932868003033088
On status #: 11
  -- id: 1256932867961126912
On status #: 12
  -- id: 1256932867772276736
On status #: 13
  -- id: 1256932867462123520
On status #: 14
  -- id: 1256932867377991681
On status #: 15
  -- id: 1256932866480472067
On status #: 16
  -- id: 1256932864748187648
O

On status #: 0
  -- id: 1256932725463715840
On status #: 1
  -- id: 1256932725358907393
On status #: 2
  -- id: 1256932724834779136
On status #: 3
  -- id: 1256932724432162816
On status #: 4
  -- id: 1256932722934640645
On status #: 5
  -- id: 1256932722083151872
On status #: 6
  -- id: 1256932721663684609
On status #: 7
  -- id: 1256932721621950466
On status #: 8
  -- id: 1256932721420431361
On status #: 9
  -- id: 1256932720959094784
On status #: 10
  -- id: 1256932719432552449
On status #: 11
  -- id: 1256932717783994368
On status #: 12
  -- id: 1256932717385486339
On status #: 13
  -- id: 1256932716538245122
On status #: 14
  -- id: 1256932714592124929
On status #: 15
  -- id: 1256932712062922752
On status #: 16
  -- id: 1256932711630921728
On status #: 17
  -- id: 1256932709017874432
On status #: 18
  -- id: 1256932708724301825
On status #: 19
  -- id: 1256932707570905095
On status #: 20
  -- id: 1256932706627186689
On status #: 21
  -- id: 1256932706371461120
On status #: 22
  --

On status #: 0
  -- id: 1256932571247546373
On status #: 1
  -- id: 1256932570953953280
On status #: 2
  -- id: 1256932570777837568
On status #: 3
  -- id: 1256932570329161728
On status #: 4
  -- id: 1256932570077396994
On status #: 5
  -- id: 1256932569960124417
On status #: 6
  -- id: 1256932569611857922
On status #: 7
  -- id: 1256932569012035590
On status #: 8
  -- id: 1256932568315748353
On status #: 9
  -- id: 1256932567594364929
On status #: 10
  -- id: 1256932567368040452
On status #: 11
  -- id: 1256932566747066368
On status #: 12
  -- id: 1256932566092767232
On status #: 13
  -- id: 1256932565694337026
On status #: 14
  -- id: 1256932565677563904
On status #: 15
  -- id: 1256932565253910529
On status #: 16
  -- id: 1256932565040029697
On status #: 17
  -- id: 1256932564930924545
On status #: 18
  -- id: 1256932564624797696
On status #: 19
  -- id: 1256932564226330625
On status #: 20
  -- id: 1256932563735773186
On status #: 21
  -- id: 1256932563337089031
On status #: 22
  --

On status #: 0
  -- id: 1256932433674420224
On status #: 1
  -- id: 1256932432281927680
On status #: 2
  -- id: 1256932431925374976
On status #: 3
  -- id: 1256932431816527873
On status #: 4
  -- id: 1256932430419685376
On status #: 5
  -- id: 1256932428737699842
On status #: 6
  -- id: 1256932428163108864
On status #: 7
  -- id: 1256932427986956288
On status #: 8
  -- id: 1256932427353616384
On status #: 9
  -- id: 1256932427181637638
On status #: 10
  -- id: 1256932426783186945
On status #: 11
  -- id: 1256932426523176962
On status #: 12
  -- id: 1256932425512308736
On status #: 13
  -- id: 1256932425487323137
On status #: 14
  -- id: 1256932422815342593
On status #: 15
  -- id: 1256932422370816000
On status #: 16
  -- id: 1256932421464817665
On status #: 17
  -- id: 1256932420248428549
On status #: 18
  -- id: 1256932419241775105
On status #: 19
  -- id: 1256932418319077377
On status #: 20
  -- id: 1256932418151305216
On status #: 21
  -- id: 1256932417815756806
On status #: 22
  --

On status #: 0
  -- id: 1256932270058823680
On status #: 1
  -- id: 1256932269945733121
On status #: 2
  -- id: 1256932269941379074
On status #: 3
  -- id: 1256932269031329792
On status #: 4
  -- id: 1256932268578226181
On status #: 5
  -- id: 1256932268561461248
On status #: 6
  -- id: 1256932267990999041
On status #: 7
  -- id: 1256932267701633024
On status #: 8
  -- id: 1256932267496140801
On status #: 9
  -- id: 1256932266392956928
On status #: 10
  -- id: 1256932265642397696
On status #: 11
  -- id: 1256932264753025024
On status #: 12
  -- id: 1256932264547475457
On status #: 13
  -- id: 1256932264379744256
On status #: 14
  -- id: 1256932262727172097
On status #: 15
  -- id: 1256932261691187205
On status #: 16
  -- id: 1256932261301112834
On status #: 17
  -- id: 1256932261087174657
On status #: 18
  -- id: 1256932261015941120
On status #: 19
  -- id: 1256932260709912578
On status #: 20
  -- id: 1256932258004430848
On status #: 21
  -- id: 1256932257819852802
On status #: 22
  --

...

On status #: 0
  -- id: 1256930183224782848
On status #: 1
  -- id: 1256930183036178432
On status #: 2
  -- id: 1256930182763474945
On status #: 3
  -- id: 1256930180511133696
On status #: 4
  -- id: 1256930179525402624
On status #: 5
  -- id: 1256930179357773825
On status #: 6
  -- id: 1256930179047256064
On status #: 7
  -- id: 1256930179039068160
On status #: 8
  -- id: 1256930178866900992
On status #: 9
  -- id: 1256930176035885056
On status #: 10
  -- id: 1256930175918338048
On status #: 11
  -- id: 1256930175662637056
On status #: 12
  -- id: 1256930175431802886
On status #: 13
  -- id: 1256930174731538433
On status #: 14
  -- id: 1256930174714724353
On status #: 15
  -- id: 1256930171808034817
On status #: 16
  -- id: 1256930171208134656
On status #: 17
  -- id: 1256930170302107649
On status #: 18
  -- id: 1256930170050502656
On status #: 19
  -- id: 1256930168901230592
On status #: 20
  -- id: 1256930168850935808
On status #: 21
  -- id: 1256930167982821377
On status #: 22
  --

On status #: 0
  -- id: 1256930031307235328
On status #: 1
  -- id: 1256930030929588226
On status #: 2
  -- id: 1256930030132719616
On status #: 3
  -- id: 1256930029272887304
On status #: 4
  -- id: 1256930029163941890
On status #: 5
  -- id: 1256930027771322368
On status #: 6
  -- id: 1256930027238756352
On status #: 7
  -- id: 1256930027062484993
On status #: 8
  -- id: 1256930026500485120
On status #: 9
  -- id: 1256930026483638274
On status #: 10
  -- id: 1256930025649004544
On status #: 11
  -- id: 1256930025581842434
On status #: 12
  -- id: 1256930025107935238
On status #: 13
  -- id: 1256930023849594881
On status #: 14
  -- id: 1256930023354830849
On status #: 15
  -- id: 1256930021937049603
On status #: 16
  -- id: 1256930021433921537
On status #: 17
  -- id: 1256930021257539584
On status #: 18
  -- id: 1256930018887970817
On status #: 19
  -- id: 1256930016920612870
On status #: 20
  -- id: 1256930014576050176
On status #: 21
  -- id: 1256930013988847622
On status #: 22
  --

On status #: 0
  -- id: 1256929880333139970
On status #: 1
  -- id: 1256929880219906048
On status #: 2
  -- id: 1256929880136019968
On status #: 3
  -- id: 1256929880119234565
On status #: 4
  -- id: 1256929879469121537
On status #: 5
  -- id: 1256929878911250432
On status #: 6
  -- id: 1256929878852526081
On status #: 7
  -- id: 1256929878441701378
On status #: 8
  -- id: 1256929877527130113
On status #: 9
  -- id: 1256929877422256129
On status #: 10
  -- id: 1256929877380358144
On status #: 11
  -- id: 1256929876130574336
On status #: 12
  -- id: 1256929874981212161
On status #: 13
  -- id: 1256929874914074624
On status #: 14
  -- id: 1256929874230431745
On status #: 15
  -- id: 1256929873412493320
On status #: 16
  -- id: 1256929871210545153
On status #: 17
  -- id: 1256929870946263045
On status #: 18
  -- id: 1256929870317150209
On status #: 19
  -- id: 1256929869180489728
On status #: 20
  -- id: 1256929868421480448
On status #: 21
  -- id: 1256929867578216450
On status #: 22
  --

On status #: 0
  -- id: 1256929727421399042
On status #: 1
  -- id: 1256929727308107777
On status #: 2
  -- id: 1256929726985207808
On status #: 3
  -- id: 1256929726322683904
On status #: 4
  -- id: 1256929724493819905
On status #: 5
  -- id: 1256929724296650755
On status #: 6
  -- id: 1256929723420196864
On status #: 7
  -- id: 1256929722077863937
On status #: 8
  -- id: 1256929720588857344
On status #: 9
  -- id: 1256929720496582657
On status #: 10
  -- id: 1256929718890196995
On status #: 11
  -- id: 1256929718550446091
On status #: 12
  -- id: 1256929718353358854
On status #: 13
  -- id: 1256929718126776322
On status #: 14
  -- id: 1256929717887746051
On status #: 15
  -- id: 1256929717032292358
On status #: 16
  -- id: 1256929716193255426
On status #: 17
  -- id: 1256929715526316032
On status #: 18
  -- id: 1256929715157229568
On status #: 19
  -- id: 1256929714603687936
On status #: 20
  -- id: 1256929713433579522
On status #: 21
  -- id: 1256929712619679746
On status #: 22
  --

On status #: 0
  -- id: 1256929582390919169
On status #: 1
  -- id: 1256929581124198401
On status #: 2
  -- id: 1256929581057011713
On status #: 3
  -- id: 1256929579798691846
On status #: 4
  -- id: 1256929579454717952
On status #: 5
  -- id: 1256929578590732289
On status #: 6
  -- id: 1256929577307234304
On status #: 7
  -- id: 1256929577110142976
On status #: 8
  -- id: 1256929575378051072
On status #: 9
  -- id: 1256929574631247873
On status #: 10
  -- id: 1256929574547410944
On status #: 11
  -- id: 1256929573284929536
On status #: 12
  -- id: 1256929573242957824
On status #: 13
  -- id: 1256929572441882624
On status #: 14
  -- id: 1256929572387295233
On status #: 15
  -- id: 1256929571946938368
On status #: 16
  -- id: 1256929571259117569
On status #: 17
  -- id: 1256929570533457922
On status #: 18
  -- id: 1256929569841414145
On status #: 19
  -- id: 1256929569115963392
On status #: 20
  -- id: 1256929568549531648
On status #: 21
  -- id: 1256929566444204032
On status #: 22
  --

In [11]:
cw.run_save()

Not empty, Successful save:  1
Not empty, Successful save:  2
Not empty, Successful save:  3
Not empty, Successful save:  4
Not empty, Successful save:  5
Not empty, Successful save:  6
Not empty, Successful save:  7
Not empty, Successful save:  8
Not empty, Successful save:  9
Not empty, Successful save:  10
Not empty, Successful save:  11
Not empty, Successful save:  12
Not empty, Successful save:  13
Not empty, Successful save:  14
Not empty, Successful save:  15
Not empty, Successful save:  16
Not empty, Successful save:  17
Not empty, Successful save:  18
Not empty, Successful save:  19
Not empty, Successful save:  20
Not empty, Successful save:  21
Not empty, Successful save:  22
Not empty, Successful save:  23
Not empty, Successful save:  24
Not empty, Successful save:  25
Not empty, Successful save:  26
Not empty, Successful save:  27
Not empty, Successful save:  28
Not empty, Successful save:  29
Not empty, Successful save:  30
Not empty, Successful save:  31
Not empty, Succes

Not empty, Successful save:  255
Not empty, Successful save:  256
Not empty, Successful save:  257
Not empty, Successful save:  258
Not empty, Successful save:  259
Not empty, Successful save:  260
Not empty, Successful save:  261
Not empty, Successful save:  262
Not empty, Successful save:  263
Not empty, Successful save:  264
Not empty, Successful save:  265
Not empty, Successful save:  266
Not empty, Successful save:  267
Not empty, Successful save:  268
Not empty, Successful save:  269
Not empty, Successful save:  270
Not empty, Successful save:  271
Not empty, Successful save:  272
Not empty, Successful save:  273
Not empty, Successful save:  274
Not empty, Successful save:  275
Not empty, Successful save:  276
Not empty, Successful save:  277
Not empty, Successful save:  278
Not empty, Successful save:  279
Not empty, Successful save:  280
Not empty, Successful save:  281
Not empty, Successful save:  282
Not empty, Successful save:  283
Not empty, Successful save:  284
Not empty,

Not empty, Successful save:  501
Not empty, Successful save:  502
Not empty, Successful save:  503
Not empty, Successful save:  504
Not empty, Successful save:  505
Not empty, Successful save:  506
Not empty, Successful save:  507
Not empty, Successful save:  508
Not empty, Successful save:  509
Not empty, Successful save:  510
Not empty, Successful save:  511
Not empty, Successful save:  512
Not empty, Successful save:  513
Not empty, Successful save:  514
Not empty, Successful save:  515
Not empty, Successful save:  516
Not empty, Successful save:  517
Not empty, Successful save:  518
Not empty, Successful save:  519
Not empty, Successful save:  520
Not empty, Successful save:  521
Not empty, Successful save:  522
Not empty, Successful save:  523
Not empty, Successful save:  524
Not empty, Successful save:  525
Not empty, Successful save:  526
Not empty, Successful save:  527
Not empty, Successful save:  528
Not empty, Successful save:  529
Not empty, Successful save:  530
Not empty,

Not empty, Successful save:  753
Not empty, Successful save:  754
Not empty, Successful save:  755
Not empty, Successful save:  756
Not empty, Successful save:  757
Not empty, Successful save:  758
Not empty, Successful save:  759
Not empty, Successful save:  760
Not empty, Successful save:  761
Not empty, Successful save:  762
Not empty, Successful save:  763
Not empty, Successful save:  764
Not empty, Successful save:  765
Not empty, Successful save:  766
Not empty, Successful save:  767
Not empty, Successful save:  768
Not empty, Successful save:  769
Not empty, Successful save:  770
Not empty, Successful save:  771
Not empty, Successful save:  772
Not empty, Successful save:  773
Not empty, Successful save:  774
Not empty, Successful save:  775
Not empty, Successful save:  776
Not empty, Successful save:  777
Not empty, Successful save:  778
Not empty, Successful save:  779
Not empty, Successful save:  780
Not empty, Successful save:  781
Not empty, Successful save:  782
Not empty,

Not empty, Successful save:  1003
Not empty, Successful save:  1004
Not empty, Successful save:  1005
Not empty, Successful save:  1006
Not empty, Successful save:  1007
Not empty, Successful save:  1008
Not empty, Successful save:  1009
Not empty, Successful save:  1010
Not empty, Successful save:  1011
Not empty, Successful save:  1012
Not empty, Successful save:  1013
Not empty, Successful save:  1014
Not empty, Successful save:  1015
Not empty, Successful save:  1016
Not empty, Successful save:  1017
Not empty, Successful save:  1018
Not empty, Successful save:  1019
Not empty, Successful save:  1020
Not empty, Successful save:  1021
Not empty, Successful save:  1022
Not empty, Successful save:  1023
Not empty, Successful save:  1024
Not empty, Successful save:  1025
Not empty, Successful save:  1026
Not empty, Successful save:  1027
Not empty, Successful save:  1028
Not empty, Successful save:  1029
Not empty, Successful save:  1030
Not empty, Successful save:  1031
Not empty, Suc

...

Not empty, Successful save:  3844
Not empty, Successful save:  3845
Not empty, Successful save:  3846
Not empty, Successful save:  3847
Not empty, Successful save:  3848
Not empty, Successful save:  3849
Not empty, Successful save:  3850
Not empty, Successful save:  3851
Not empty, Successful save:  3852
Not empty, Successful save:  3853
Not empty, Successful save:  3854
Not empty, Successful save:  3855
Not empty, Successful save:  3856
Not empty, Successful save:  3857
Not empty, Successful save:  3858
Not empty, Successful save:  3859
Not empty, Successful save:  3860
Not empty, Successful save:  3861
Not empty, Database already have document: 1256930031307235328
Not empty, Successful save:  3862
Not empty, Successful save:  3863
Not empty, Successful save:  3864
Not empty, Successful save:  3865
Not empty, Successful save:  3866
Not empty, Successful save:  3867
Not empty, Successful save:  3868
Not empty, Successful save:  3869
Not empty, Successful save:  3870
Not empty, Successf

Not empty, Successful save:  4082
Not empty, Successful save:  4083
Not empty, Successful save:  4084
Not empty, Successful save:  4085
Not empty, Successful save:  4086
Not empty, Successful save:  4087
Not empty, Successful save:  4088
Not empty, Successful save:  4089
Not empty, Successful save:  4090
Not empty, Successful save:  4091
Not empty, Successful save:  4092
Not empty, Successful save:  4093
Not empty, Successful save:  4094
Not empty, Successful save:  4095
Not empty, Successful save:  4096
Not empty, Successful save:  4097
Not empty, Successful save:  4098
Not empty, Successful save:  4099
Not empty, Successful save:  4100
Not empty, Successful save:  4101
Not empty, Successful save:  4102
Not empty, Successful save:  4103
Not empty, Successful save:  4104
Not empty, Successful save:  4105
Not empty, Successful save:  4106
Not empty, Successful save:  4107
Not empty, Successful save:  4108
Not empty, Successful save:  4109
Not empty, Successful save:  4110
Not empty, Suc

Not empty, Successful save:  4323
Not empty, Successful save:  4324
Not empty, Successful save:  4325
Not empty, Successful save:  4326
Not empty, Successful save:  4327
Not empty, Successful save:  4328
Not empty, Successful save:  4329
Not empty, Successful save:  4330
Not empty, Successful save:  4331
Not empty, Successful save:  4332
Not empty, Successful save:  4333
Not empty, Successful save:  4334
Not empty, Successful save:  4335
Not empty, Successful save:  4336
Not empty, Successful save:  4337
Not empty, Successful save:  4338
Not empty, Successful save:  4339
Not empty, Successful save:  4340
Not empty, Successful save:  4341
Not empty, Successful save:  4342
Not empty, Successful save:  4343
Not empty, Successful save:  4344
Not empty, Successful save:  4345
Not empty, Successful save:  4346
Not empty, Successful save:  4347
Not empty, Successful save:  4348
Not empty, Successful save:  4349
Not empty, Successful save:  4350
Not empty, Successful save:  4351
Not empty, Suc

Not empty, Successful save:  4562
Not empty, Successful save:  4563
Not empty, Successful save:  4564
Not empty, Successful save:  4565
Not empty, Successful save:  4566
Not empty, Successful save:  4567
Not empty, Successful save:  4568
Not empty, Successful save:  4569
Not empty, Successful save:  4570
Not empty, Successful save:  4571
Not empty, Successful save:  4572
Not empty, Successful save:  4573
Not empty, Successful save:  4574
Not empty, Successful save:  4575
Not empty, Successful save:  4576
Not empty, Successful save:  4577
Not empty, Successful save:  4578
Not empty, Successful save:  4579
Not empty, Successful save:  4580
Not empty, Successful save:  4581
Not empty, Successful save:  4582
Not empty, Successful save:  4583
Not empty, Successful save:  4584
Not empty, Successful save:  4585
Not empty, Successful save:  4586
Not empty, Successful save:  4587
Not empty, Successful save:  4588
Not empty, Successful save:  4589
Not empty, Successful save:  4590
Not empty, Suc

KeyboardInterrupt: 

In [6]:
queue.empty()

True

---
#### api

In [ ]:
consumer_key="dL2ZAItvpFLiCULxa8Y7E5Utg"
consumer_secret="hnLfXSmCTNSAJ2rsU9QOAvUFG159wysgGwy38sQyt0XKDte70M"

access_token="709584127-zufPerRgSs4TDMscyfon47CIsN2AuXapBU8XEhty"
access_token_secret="wzpAq42SKJP4XpqXqWAfiNsAJmXKjN2AQSeP51BvIETE2"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)

In [ ]:
api.search('stayhome')

---
#### os

In [ ]:
import os
os.getcwd()
os.remove('log_tweetWorker.txt')

In [ ]:
# List dir
os.listdir('.')

---


In [ ]:
# For OAuth <-- authentication protocol
consumer_key="dL2ZAItvpFLiCULxa8Y7E5Utg"
consumer_secret="hnLfXSmCTNSAJ2rsU9QOAvUFG159wysgGwy38sQyt0XKDte70M"

access_token="709584127-zufPerRgSs4TDMscyfon47CIsN2AuXapBU8XEhty"
access_token_secret="wzpAq42SKJP4XpqXqWAfiNsAJmXKjN2AQSeP51BvIETE2"

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = API(auth)

In [ ]:
# Helper function to save data into a JSON file
# file_name: the file name of the data on Google Drive
# file_content: the data you want to save
def save_json(file_name, file_content):
    with open(path + file_name, 'w', encoding='utf-8') as f:
        json.dump(file_content, f, ensure_ascii=False, indent=4)

In [ ]:
# Helper function to handle twitter API rate limit
def limit_handled(cursor, list_name):
    while True:
        try:
            yield cursor.next()
            
        # Catch Twitter API rate limit exception and wait for 15 minutes
        except tweepy.RateLimitError:
            print("\nData points in list = {}".format(len(list_name)))
            print('Hit Twitter API rate limit.')
            for i in range(3, 0, -1):
                print("Wait for {} mins.".format(i * 5))
                time.sleep(5 * 60)
                
        # Catch any other Twitter API exceptions
        except tweepy.error.TweepError:
            print('\nCaught TweepError exception' )

In [ ]:
# Helper function to get all tweets of a specified user
# NOTE:This method only allows access to the most recent 3200 tweets
# Source: https://gist.github.com/yanofsky/5436496
def get_all_tweets(screen_name):
    # initialize a list to hold all the Tweets
    alltweets = []
    # make initial request for most recent tweets 
    # (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    # save most recent tweets
    alltweets.extend(new_tweets)
    # save the id of the oldest tweet less one to avoid duplication
    oldest = alltweets[-1].id - 1
    # keep grabbing tweets until there are no tweets left
    while len(new_tweets) > 0:
        print("getting tweets before %s" % (oldest))
        # all subsequent requests use the max_id param to prevent
        # duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        # save most recent tweets
        alltweets.extend(new_tweets)
        # update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
        print("...%s tweets downloaded so far" % (len(alltweets)))
        ### END OF WHILE LOOP ###
    # transform the tweepy tweets into a 2D array that will 
    # populate the csv
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text, tweet.favorite_count,tweet.in_reply_to_screen_name, tweet.retweeted] for tweet in alltweets]
    # write the csv
    with open(path + '%s_tweets.csv' % screen_name, 'w') as f:
    writer = csv.writer(f)
        writer.writerow(["id","created_at","text","likes","in reply to","retweeted"])
        writer.writerows(outtweets)
    pass

In [ ]:
count = 0
for status in tweepy.Cursor(api.home_timeline).items():
    count += 1
    print(status, '\n'*10)
    
print(count)

In [ ]:
count

In [ ]:
api.home_timeline()